```text
TECHNICAL PROCESSES

temporarily split into 3 blocks:
    downloading and processing data by running a batch script
    loading processed ready-to-use csv files
    setting up the dataframes necessary for further analysis and visualization


THIS IS ONLY A REFORMATTING SUGGESTION, WHICH AIMS TO MAKE THE NOTEBOOK MORE READABLE AND STREAMLINED FOR THE USER
IT IS NOT THE UPDATE OF THE PREVIOUS PROJECT NOTEBOOK FILE, BUT MERELY A PROPOSED ALTERNATIVE, THEREFORE IT HAS BEEN COMMITTED AS A SEPARATE FILE
THE DATA ANALYSIS / VISUALIZATION HAS NOT BEEN CHANGED, EXCEPT FOR UPDATING CERTAIN VARIABLE NAMES, BUT FUNCTIONALITY HAS REMAINED UNCHANGED
```

In [103]:
import subprocess
import pandas as pd
import os
from datetime import datetime

# updating and processing data
subprocess.run(["cmd","/c","update_and_process_data.bat"])


CompletedProcess(args=['cmd', '/c', 'update_and_process_data.bat'], returncode=0)

In [104]:
# loading csv files
data = {}
for name in ["stops_processed.csv", "routes_processed.csv", "trips_processed.csv", "stop_times_processed.csv", "calendar_processed.csv", "calendar_dates_processed.csv", "transfers_processed.csv"]:
    df = pd.read_csv(os.path.join("data/processed",name),low_memory=False)
    name = name[:-4]
    data[name] = df

In [105]:
# setting up dataframes

stops_df          = data["stops_processed"]
routes_df         = data["routes_processed"]
trips_df          = data["trips_processed"]
stop_times_df     = data["stop_times_processed"]
calendar_df       = data["calendar_processed"]
calendar_dates_df = data["calendar_dates_processed"]
transfers_df      = data["transfers_processed"]

# merging dataframes

route_type_df     = routes_df[["route_type", "route_id"]]
route_type_df     = pd.merge(route_type_df, trips_df[['route_id', 'trip_id']], on='route_id')
route_type_df     = pd.merge(route_type_df, stop_times_df[['stop_id', 'trip_id']], on='trip_id')
route_type_df     = pd.merge(route_type_df, stops_df[['stop_id', 'stop_name']], on='stop_id')
trips_trans       = pd.merge(trips_df, routes_df[["route_type", "route_id"]], on = "route_id")
stop_times_trans  = pd.merge(stop_times_df, trips_trans[["route_type", "trip_id"]], on = "trip_id")
stop_trans        = pd.merge(stops_df, stop_times_trans[["route_type", "stop_id"]], on = "stop_id")



```text
HIGH VISIBILITY
```

In [106]:
# high-visibility dataframes

trips_count = pd.DataFrame(stop_times_df.groupby('stop_id')['trip_id'].nunique())
trips_count


,trip_id
stop_id,
T53041,85
T53047,476
T53051,32
T53068,44
T53097,60
...,...
U999Z2P,84
U99Z1P,643
U99Z2P,629


In [107]:
high_visibility = stops_df[["stop_id", "stop_name"]]
high_visibility_merged = pd.merge(high_visibility, trips_count, on='stop_id')
# high_visibility_merged = (high_visibility_merged.groupby(["stop_id", "stop_name"])["trip_id"].sum().sort_values(ascending=False).reset_index())
high_visibility_merged = high_visibility_merged.groupby("stop_name")["trip_id"].sum().sort_values(ascending=False).reset_index()
high_visibility_merged.head(10)

,stop_name,trip_id
0,Želivského,7594
1,Smíchovské nádraží,6981
2,Anděl,6954
3,Palmovka,5970
4,Lihovar,5886
5,Kobylisy,5886
6,Černý Most,5769
7,Karlovo náměstí,5476
8,Letňany,5385
9,Florenc,5190


In [108]:
# trying to sort it per route type
high = pd.merge(high_visibility_merged, route_type_df[["stop_name", "route_type"]], on = "stop_name")
high = high.drop_duplicates()
high.head(30)

,stop_name,trip_id,route_type
0,Želivského,7594,1
1378,Želivského,7594,0
4375,Želivského,7594,3
7594,Smíchovské nádraží,6981,1
8563,Smíchovské nádraží,6981,0
9411,Smíchovské nádraží,6981,3
14575,Anděl,6954,1
15543,Anděl,6954,0
19868,Anděl,6954,3
21529,Palmovka,5970,1


```text
LONGEST EXPOSURE

1. longest amount of time
```

In [109]:
def time_to_seconds(t):
    h, m, s = map(int, t.split(':'))
    return h*3600 + m*60 + s

def seconds_to_time(seconds):
    # Convert to integer (in case it's float)
    seconds = int(seconds)
    
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    secs = seconds % 60
    
    # Format as HH:MM:SS with leading zeros
    return f"{hours:02d}:{minutes:02d}:{secs:02d}"

```text
- groupby trip_id
- find the first and last row per unique trip_id
- convert to seconds
- find the difference
- average per route_id
```

In [110]:
## defining first rows and compusing seconds ##
first_rows = stop_times_df.loc[stop_times_df.groupby('trip_id')['arrival_time'].idxmin()]

first_rows = first_rows[["trip_id", "arrival_time"]]
first_rows['arrival_seconds'] = (
    first_rows['arrival_time']
    .apply(time_to_seconds)
)

In [111]:
## defining last rows and computing seconds ## 
last_rows  = stop_times_df.loc[stop_times_df.groupby('trip_id')['arrival_time'].idxmax()]

last_rows = last_rows[["trip_id", "arrival_time"]]
last_rows['arrival_seconds'] = (
    last_rows['arrival_time']
    .apply(time_to_seconds)
)

In [112]:
## computing duration per trip_id ##
duration_series = (last_rows.set_index("trip_id")["arrival_seconds"] - first_rows.set_index("trip_id")["arrival_seconds"])

duration = duration_series.reset_index()  # trip_id becomes a column
duration = duration.rename(columns={"arrival_seconds": "duration_seconds"})

duration = pd.merge(duration, trips_df[['trip_id', 'route_id']], on='trip_id')

In [113]:
## finding average per route ##
avg_duration_per_route = (duration.groupby('route_id')['duration_seconds'].mean().reset_index())

avg_duration_per_route['duration_hms'] = avg_duration_per_route['duration_seconds'].apply(seconds_to_time)
avg_duration_per_route = pd.merge(avg_duration_per_route, routes_df[['route_short_name', 'route_id']], on='route_id')

avg_duration_per_route = avg_duration_per_route.sort_values(
    by='duration_seconds',
    ascending=False
).reset_index(drop=True)

In [114]:
avg_duration_per_route.head()

,route_id,duration_seconds,duration_hms,route_short_name
0,L1106,9210.000000,02:33:30,T6
1,L730,8796.000000,02:26:36,730
2,L416,8700.000000,02:25:00,416
3,L700,8592.857143,02:23:12,700
4,L1110,8145.000000,02:15:45,T10


```text
2. highest amount of stops
```

In [115]:
## counting stops per trip ##
stops_counts = stop_times_df.groupby("trip_id")['stop_id'].count().reset_index(name='num_stops').sort_values(by='num_stops', ascending=False)

stops_counts = pd.merge(stops_counts, trips_df[['trip_id', 'route_id']], on='trip_id')

In [116]:
## computing average number of stops per route ##
avg_stops_per_route = (stops_counts.groupby('route_id')['num_stops'].mean().round(0).reset_index())

avg_stops_per_route = pd.merge(avg_stops_per_route, routes_df[['route_short_name', 'route_id']], on='route_id')
avg_stops_per_route = avg_stops_per_route.sort_values(
    by='num_stops',
    ascending=False
).reset_index(drop=True)

In [117]:
avg_stops_per_route.head()

,route_id,num_stops,route_short_name
0,L908,60.0,908
1,L910,58.0,910
2,L911,54.0,911
3,L913,52.0,913
4,L617,52.0,617


```text
HIGHEST REPETITION
```

In [118]:
repetition = pd.DataFrame(trips_df.groupby('route_id')['trip_id'].nunique())

In [119]:
repetition

,trip_id
route_id,
L1,355
L10,351
L100,162
L1004,50
L1005,2
...,...
L98,37
L99,21
L991,1378


In [120]:
high_repetition = routes_df[["route_id", "route_short_name"]]

high_repetition = pd.merge(high_repetition, repetition, on='route_id').sort_values(by='trip_id', ascending=False).reset_index(drop=True)
high_repetition.head(10)

,route_id,route_short_name,trip_id
0,L991,A,1378
1,L993,C,1022
2,L992,B,969
3,L812,X12,954
4,L22,22,771
5,L17,17,717
6,L59,59,708
7,L9,9,699
8,L137,137,589
9,L5,5,581


## TO DO

1. __high visibility:__
- sort by route type -> when one stop has metro, tram, and bus, do we just say that it's metro
- in other words, do we sort by metro > tram > trolley > bus (?)

2. __longest exposure__
- do we sort by route type, since now trains have the longest duration
- or just make exclude regional routes outside of Prague city (or if the route is at night)
- for the most amount of stops, do we once again sort by route type, or not

3. __highest repetition__
- do we sort for weekdays, weekends, full weeks
- and do we sort by route type

```text
my opinions:
1. i think the proposed [metro > tram > trolley > bus] system makes sense, but i would have to look into it a bit more, probably on saturday
2. i think we should exclude regional routes. i will look into doing that during processing, so we do not need to worry about it in the notebook file, i will report back tomorrow
3. i think we should sort by all weekdays combined as one output, by weekends as a second output and full weeks as a third output. 

- concerning sorting by route type, i think its an important distinction and it would probably be useful to provide separate outputs for each, both in longest exposure and highest repetition
```